In [2]:
import pulp
import pandas as pd

In [3]:
#create unpack function to extract relevant info to dataframe. then pass in each series or make indv dataframe as function params

In [21]:
def optimize_cardspace(cards, fees, spending, score):
    """
    cards[card] = {"min_score": int, "rates": {category: rate}}
    fees[card]  = annual fee
    spending[category] = spend in that category
    """
    # NOTE THAT FUNCTIONALITY FOR TRIGGER BASED REWARDS IS NOT SUPPORTED

    prob = pulp.LpProblem("Maximize_Rewards", pulp.LpMaximize)
    cats = list(spending.keys())
    card_list = list(cards.keys())

    #eligibility filter from score
    eligible = {c: 1 if score >= cards[c].get("min_score", 0) else 0 for c in card_list}

    #decision vars

    #card selection indicator
    y = {c: pulp.LpVariable(f"hold_{c}", 0, 1, pulp.LpBinary) for c in card_list}

    #category decision 
    x = {(c,k): pulp.LpVariable(f"use_{c}_{k}", 0, 1, pulp.LpBinary)
         for c in card_list for k in cats}

    #objective: rewards - fees
    reward = pulp.lpSum(spending[k] * cards[c]["rates"].get(k, 0.0) * x[(c,k)] for c in card_list for k in cats)
    fee = pulp.lpSum(fees[c] * y[c] for c in card_list)
    prob += reward - fee

    #at most one card per category
    for k in cats:
        prob += pulp.lpSum(x[(c,k)] for c in card_list) <= 1

    #can only use a card if you hold it
    for c in card_list:
        for k in cats:
            prob += x[(c,k)] <= y[c]
            

    for c in card_list:
        #can only hold if eligible by score
        prob += y[c] <= eligible[c]
        # If a card is held, it must be used in at least one category. (Avoids treating 0 fee cards as free)
        prob += pulp.lpSum(x[(c,k)] for k in cats) >= y[c]


    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    if pulp.LpStatus[pulp.LpStatusOptimal] != 'Optimal' and pulp.LpStatus[prob.status] != 'Optimal':
        return "No optimal solution found", 0.0, set(), {}

    total = pulp.value(reward - fee)
    chosen = {k: max(card_list, key=lambda c: pulp.value(x[(c,k)])) for k in cats}
    held   = {c for c in card_list if pulp.value(y[c]) > 0.5}
    return chosen, total, held


In [24]:
#Test case: normal
cards1 = {
    "A": {"min_score": 680, "rates": {"groceries": 0.03, "travel": 0.01}},
    "B": {"min_score": 700, "rates": {"groceries": 0.01, "travel": 0.02}},
}
fees1 = {"A": 95, "B": 95}
spending1 = {"groceries": 5000, "travel": 4000}
score1 = 720

#should just hold A bc pnl for B is 80-95=-15.

#Test case: zero spend
cards2 = {
    "A": {"min_score": 680, "rates": {"groceries": 0.04, "dining": 0.01}},
    "B": {"min_score": 650, "rates": {"groceries": 0.015, "dining": 0.015}},
}
fees2 = {"A": 95, "B": 0}
spending2 = {"groceries": 3000, "dining": 0}
score2 = 700

#should only hold B bc pnl for A is negative

# Test case: High fee dominates
cards3 = {
    "A": {"min_score": 740, "rates": {"travel": 0.05}},
    "B": {"min_score": 680, "rates": {"travel": 0.02}},
}
fees3 = {"A": 50, "B": 0}
spending3 = {"travel": 2000}
score3 = 800

#Gain 40 for holding B, gain 50 for holding A. Should hold both if B is free. 

# Test case: Inelligible by score
cards4 = {
    "A": {"min_score": 760, "rates": {"gas": 0.04}},
    "B": {"min_score": 600, "rates": {"gas": 0.01}},
}
fees4 = {"A": 0, "B": 0}
spending4 = {"gas": 1000}
score4 = 500

#Test case: Tie
cards5 = {
    "A": {"min_score": 650, "rates": {"dining": 0.03}},
    "B": {"min_score": 650, "rates": {"dining": 0.03}},
}
fees5 = {"A": 0, "B": 0}
spending5 = {"dining": 2500}
score5 = 700

#Test case: Strictly Negative
cards6 = {
    "A": {"min_score": 650, "rates": {"dining": 0.03}},
    "B": {"min_score": 650, "rates": {"dining": 0.03}},
}
fees6 = {"A": 10000, "B": 10000}
spending6 = {"dining": 2500}
score6 = 700

In [25]:
def run_case(n, cards, fees, spending, score):
    print(f"\nCase {n}")
    chosen, total, held = optimize_cardspace(cards, fees, spending, score)
    print("Held:", held)
    print("Assignments:", chosen)
    print("Net reward ($):", round(total, 2))

for i, data in enumerate([
    (cards1, fees1, spending1, score1),
    (cards2, fees2, spending2, score2),
    (cards3, fees3, spending3, score3),
    (cards4, fees4, spending4, score4),
    (cards5, fees5, spending5, score5),
    (cards6, fees6, spending6, score6)
], start=1):
    run_case(i, *data)


Case 1
Held: {'A'}
Assignments: {'groceries': 'A', 'travel': 'A'}
Net reward ($): 95.0

Case 2
Held: {'B'}
Assignments: {'groceries': 'B', 'dining': 'A'}
Net reward ($): 45.0

Case 3
Held: {'A'}
Assignments: {'travel': 'A'}
Net reward ($): 50.0

Case 4
Held: set()
Assignments: {'gas': 'A'}
Net reward ($): 0.0

Case 5
Held: {'A'}
Assignments: {'dining': 'A'}
Net reward ($): 75.0

Case 6
Held: set()
Assignments: {'dining': 'A'}
Net reward ($): 0.0
